In [163]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

In [164]:
# import file
etfFiles=pd.read_csv('justetf 3.csv')
etfFiles

,isin,wkn,name,fundProvider,legalStructure,quote,quote52Low,quote52High,ytdReturnCUR,ter,...,exposureCountry_Argentina,exposureCountry_Macedonia,exposureCountry_Peru,exposureCountry_Russia,exposureCountry_Cyprus,exposureCountry_Colombia,exposureCountry_Czech Republic,exposureCountry_Slovakia,exposureCountry_Slovenia,exposureCountry_Bulgaria
0,IE00B0M62Y33,A0HGWF,iShares AEX UCITS ETF,iShares,ETF,73.98,62.81,76.92,0.0541,0.0030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,5.80,7.13,0.0503,0.0030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,63.16,77.27,0.0510,0.0030,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IE000RN036E0,A3DGK2,First Trust Alerian Disruptive Technology Real...,First Trust,ETF,17.44,17.07,23.80,0.0029,0.0060,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.7,10.39,13.12,-0.0599,0.0040,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2259,IE00BD6RZW23,A2AS6C,WisdomTree US Equity Income UCITS ETF EUR Hedg...,WisdomTree,ETF,19.69,18.67,22.28,-0.0777,0.0035,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2260,IE00BZ56RG20,A2AGPV,WisdomTree US Quality Dividend Growth UCITS ET...,WisdomTree,ETF,31.98,29.23,34.89,0.0019,0.0033,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2261,IE00B3Y8D011,A1C1G8,Xtrackers Portfolio Income UCITS ETF 1D,Xtrackers,ETF,11.79,11.41,12.92,0.0164,0.0065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2262,CH0496484640,A22FMC,21Shares Bitcoin Suisse Index ETP,21Shares,ETN,11.88,7.35,20.93,0.6027,0.0250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Data Cleaning

In [165]:
#Drop the columns that will not be used

etfFiles.drop(['securitiesLending','wkn','ticker','fiveYearVolatilityCUR','fiveYearReturnPerRiskCUR','fiveYearReturnCUR','yearDividendYield'], axis=1,inplace=True)

In [166]:
#Data conversion 

etfFiles['inceptionDate'] = pd.to_datetime(etfFiles['inceptionDate'], errors='coerce')
etfFiles['quoteDate'] = pd.to_datetime(etfFiles['quoteDate'])
etfFiles['ter'] = pd.to_numeric(etfFiles['ter'], errors='coerce')
etfFiles['quote']=pd.to_numeric(etfFiles['quote'],errors='coerce')
etfFiles['fundSizeMillions']=pd.to_numeric(etfFiles['fundSizeMillions'],errors='coerce') 


In [167]:
# Filter quoteDate of the ETFs - March 2023

filtered_data = etfFiles[(etfFiles['quoteDate'] >= '2023-03-01') & (etfFiles['quoteDate'] < '2023-04-01')]
filtered_data

#filtered_data['quoteDate'].unique()
#filtered_data['quoteDate'].unique()
#filtered_data['quoteDate'].unique()


,isin,name,fundProvider,legalStructure,quote,quote52Low,quote52High,ytdReturnCUR,ter,yearVolatilityCUR,...,exposureCountry_Argentina,exposureCountry_Macedonia,exposureCountry_Peru,exposureCountry_Russia,exposureCountry_Cyprus,exposureCountry_Colombia,exposureCountry_Czech Republic,exposureCountry_Slovakia,exposureCountry_Slovenia,exposureCountry_Bulgaria
0,IE00B0M62Y33,iShares AEX UCITS ETF,iShares,ETF,73.98,62.81,76.92,0.0541,0.0030,0.1879,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,IE00BMTX2B82,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,5.80,7.13,0.0503,0.0030,0.1906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NL0009272749,VanEck AEX UCITS ETF,VanEck,ETF,72.34,63.16,77.27,0.0510,0.0030,0.1898,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IE000RN036E0,First Trust Alerian Disruptive Technology Real...,First Trust,ETF,17.44,17.07,23.80,0.0029,0.0060,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IE00BKPTXQ89,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,10.39,13.12,-0.0599,0.0040,0.2473,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2259,IE00BD6RZW23,WisdomTree US Equity Income UCITS ETF EUR Hedg...,WisdomTree,ETF,19.69,18.67,22.28,-0.0777,0.0035,0.2135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2260,IE00BZ56RG20,WisdomTree US Quality Dividend Growth UCITS ET...,WisdomTree,ETF,31.98,29.23,34.89,0.0019,0.0033,0.2117,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2261,IE00B3Y8D011,Xtrackers Portfolio Income UCITS ETF 1D,Xtrackers,ETF,11.79,11.41,12.92,0.0164,0.0065,0.0650,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2262,CH0496484640,21Shares Bitcoin Suisse Index ETP,21Shares,ETN,11.88,7.35,20.93,0.6027,0.0250,0.6274,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
# index values
filtered_data.index = filtered_data['isin'].values
#filtered_data.set_index('isin',inplace=True)

In [169]:
# Delete values where the ytdReturnCUR  or yearVolatilityCUR is null
filtered_data.dropna(subset=['ytdReturnCUR'],inplace=True)
filtered_data.dropna(subset=['yearVolatilityCUR'],inplace=True)

/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/1995893823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.dropna(subset=['ytdReturnCUR'],inplace=True)
/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/1995893823.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.dropna(subset=['yearVolatilityCUR'],inplace=True)


Data Transformation

In [177]:
#Groups of ETF according to the ytdReturnCUR

filtered_data['Group']='A'
filtered_data.loc[(filtered_data['ytdReturnCUR']>=0.04)&(filtered_data['ytdReturnCUR']<0.06),'Group']='B'
filtered_data.loc[(filtered_data['ytdReturnCUR']>=0.06)&(filtered_data['ytdReturnCUR']<0.08),'Group']='C'
filtered_data.loc[(filtered_data['ytdReturnCUR']>=0.08)&(filtered_data['ytdReturnCUR']<0.14),'Group']='D'
filtered_data.loc[(filtered_data['ytdReturnCUR']>0.14),'Group']='E'
filtered_data.loc[(filtered_data['ytdReturnCUR']>0.14),'Group']='E'

/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/3300265442.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Group']='A'


In [179]:
# check distribution by groups
filtered_data.groupby('Group').ytdReturnCUR.aggregate([np.count_nonzero, np.min, np.max, np.mean])


,count_nonzero,amin,amax,mean
Group,,,,
A,1448,-0.2202,0.1400,0.001267
B,219,0.0400,0.0599,0.048426
C,104,0.0600,0.0798,0.070437
D,169,0.0800,0.1398,0.102924
E,89,0.1401,1.4638,0.411524


In [180]:
# What is the equivalency of ter on the quote
filtered_data['Fee_comparison']=(filtered_data['ter']<0.0005*filtered_data['quote']).astype(int)

/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/2948914323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Fee_comparison']=(filtered_data['ter']<0.0005*filtered_data['quote']).astype(int)


In [181]:
# column "Currency_Risk". If the colum " currencyRisk" is hedge is good=1 else 0
filtered_data['Currency_Risk'] = filtered_data['currencyRisk'].apply(lambda x: 1 if x == 'Currency hedged' else 0)

#filtered_data['Currency_Risk'].unique()
#filtered_data['currencyRisk'].unique()

/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/473497394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Currency_Risk'] = filtered_data['currencyRisk'].apply(lambda x: 1 if x == 'Currency hedged' else 0)


In [182]:
#"Compliance" . If the column "UCITSCompliance" is yes then 1 else 0

filtered_data['Compliance'] = filtered_data['UCITSCompliance'].apply(lambda x: 1 if x == 'Yes' else 0)

#filtered_data['Compliance'].unique()
#filtered_data['UCITSCompliance'].unique()

/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/2576993125.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Compliance'] = filtered_data['UCITSCompliance'].apply(lambda x: 1 if x == 'Yes' else 0)


In [183]:
#Volume" if the column "fundSizeMillions" is more than 10 Million then  good=1 else 0

filtered_data['Volume'] = filtered_data['fundSizeMillions'].apply(lambda x:1 if x > 10 else 0)

#filtered_data['fundSizeMillions'].info()
#filtered_data['Volume'].unique()

/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/1801791253.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Volume'] = filtered_data['fundSizeMillions'].apply(lambda x:1 if x > 10 else 0)


In [184]:
# K-means to get 3 risk groups - Conservative, Moderate and Aggressive

kmeans = KMeans(n_clusters=3, random_state=0).fit(filtered_data[['yearVolatilityCUR']])
filtered_data['Risk_Group'] = kmeans.labels_

/Users/pinkman420/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/3698413111.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Risk_Group'] = kmeans.labels_


In [185]:
# check the values of the distribution to assign the risk group 

filtered_data.groupby('Risk_Group').yearVolatilityCUR.aggregate([np.count_nonzero, np.min, np.max, np.mean])


,count_nonzero,amin,amax,mean
Risk_Group,,,,
0,978,0.1836,0.5193,0.241418
1,999,0.0011,0.1834,0.126305
2,59,0.5810,1.9948,0.837736


In [186]:
# assign the risk group 

filtered_data['Risk_Group']= filtered_data['Risk_Group'].map({0: 'Moderate', 1: 'Conservative', 2: 'Aggressive'})

/var/folders/fm/_0pxqhs90jz9npd2zkv3vn7m0000gn/T/ipykernel_98056/1931311246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Risk_Group']= filtered_data['Risk_Group'].map({0: 'Moderate', 1: 'Conservative', 2: 'Aggressive'})


In [187]:
# check values of risk group

filtered_data.groupby('Risk_Group').yearVolatilityCUR.aggregate([np.count_nonzero, np.min, np.max, np.mean])


,count_nonzero,amin,amax,mean
Risk_Group,,,,
Aggressive,59,0.5810,1.9948,0.837736
Conservative,999,0.0011,0.1834,0.126305
Moderate,978,0.1836,0.5193,0.241418


In [188]:
print(filtered_data)

                      isin                                               name  \
IE00B0M62Y33  IE00B0M62Y33                              iShares AEX UCITS ETF   
IE00BMTX2B82  IE00BMTX2B82                    iShares AEX UCITS ETF EUR (Acc)   
NL0009272749  NL0009272749                               VanEck AEX UCITS ETF   
IE00BKPTXQ89  IE00BKPTXQ89  HANetf Alerian Midstream Energy Dividend UCITS...   
CH1146882316  CH1146882316                              21Shares Algorand ETP   
...                    ...                                                ...   
IE00BD6RZW23  IE00BD6RZW23  WisdomTree US Equity Income UCITS ETF EUR Hedg...   
IE00BZ56RG20  IE00BZ56RG20  WisdomTree US Quality Dividend Growth UCITS ET...   
IE00B3Y8D011  IE00B3Y8D011            Xtrackers Portfolio Income UCITS ETF 1D   
CH0496484640  CH0496484640                  21Shares Bitcoin Suisse Index ETP   
CH0445689208  CH0445689208                   21Shares Crypto Basket Index ETP   

             fundProvider l